In [1]:
from __future__ import print_function
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import contrib 
import pandas as pd
from tensorflow.python.framework import ops
import urllib
import random
from math import exp
from math import log
import scipy
import IPython
import sys
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
sess = tf.Session(config=config) 
K.set_session(sess)

%matplotlib inline

Using TensorFlow backend.


In [2]:
# Format data
with tf.device('/gpu:0'):
    def format_data(filename):
        training = open(filename)
        header = training.readline()
        fields = header.strip().replace('"','').split(',')
        featureNames = fields[:-1]
        labelName = fields[-1]
        X = []
        Y = []
        prev_one = [float(x) for x in training.readline().split(',')]
        prev_two = [float(x) for x in training.readline().split(',')]
        prev_three = [float(x) for x in training.readline().split(',')]
        prev_four = [float(x) for x in training.readline().split(',')]
        prev_five = [float(x) for x in training.readline().split(',')]
        y_line = [float(x) for x in training.readline().split(',')]
        for l in training:
            prev_flights = np.vstack((prev_one,prev_two,prev_three,prev_four,prev_five))
            X.append(prev_flights)
            Y.append(y_line[-1])
            prev_one = prev_two
            prev_two = prev_three
            prev_three = prev_four
            prev_four = prev_five
            prev_five = y_line
            y_line = [float(x) for x in l.split(',')]
            
        X = np.array(X)
        Y = np.array(Y)
        Y = Y.astype(int)
        nb_classes = 15
        Y = np.eye(nb_classes)[Y]
        X = X
        Y = np.roll(Y,2)

        return X, Y

In [ ]:
# Import and Format Training Set
X_train, Y_train = format_data('training.csv')
X_train = X_train.reshape((X_train.shape[0], 5, 13, 1))

# Import and Format Dev Set
X_dev, Y_dev = format_data('dev.csv')
X_dev = X_dev.reshape((X_dev.shape[0], 5, 13, 1))

num_flights = 5

In [4]:
# Create Placeholder needed for latter use
def create_placeholders(n_H0, n_W0, n_C0, n_y):

    X = tf.placeholder(tf.float32, shape=(None, n_H0, n_W0, n_C0), name = "X")
    Y = tf.placeholder(tf.float32, shape=(None, n_y), name = "Y")
    
    return X, Y

In [5]:
# Initialize necessary parameters for CNN
def initialize_parameters():

    W1 = tf.get_variable("W1", [num_flights, num_flights, 1, 16], initializer = tf.contrib.layers.xavier_initializer())
    W2 = tf.get_variable("W2", [num_flights, num_flights, 16, 32], initializer = tf.contrib.layers.xavier_initializer())
    W3 = tf.get_variable("W3", [num_flights, num_flights, 32, 64], initializer = tf.contrib.layers.xavier_initializer())
    W4 = tf.get_variable("W4", [num_flights, num_flights, 64, 128], initializer = tf.contrib.layers.xavier_initializer())
    W5 = tf.get_variable("W5", [num_flights, num_flights, 128, 256], initializer = tf.contrib.layers.xavier_initializer())
    
    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3,
                  "W4": W4,
                  "W5": W5}
    
    return parameters

In [6]:
# Forward_propagation Function

def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    W5 = parameters['W5']
    

    Z1 = tf.nn.conv2d(X,W1, strides = [1,1,1,1], padding = 'SAME')

    A1 =  tf.nn.relu(Z1)

    P1 = tf.nn.max_pool(A1, ksize = [1,num_flights,num_flights,1], strides = [1,1,1,1], padding = 'SAME')

    Z2 = tf.nn.conv2d(A1,W2, strides = [1,1,1,1], padding = 'SAME')

    A2 = tf.nn.relu(Z2)

    P2 = tf.nn.max_pool(A2, ksize = [1,num_flights,num_flights,1], strides = [1,1,1,1], padding = 'SAME')
    
    Z3 = tf.nn.conv2d(P2,W3, strides = [1,1,1,1], padding = 'SAME')
    
    A3 = tf.nn.relu(Z3)
    
    P3 = tf.nn.max_pool(A3, ksize = [1,num_flights,num_flights,1], strides = [1,1,1,1], padding = 'SAME')
    
    Z4 = tf.nn.conv2d(P3,W4, strides = [1,1,1,1], padding = 'SAME')
    
    A4 = tf.nn.relu(Z4)
    
    P4 = tf.nn.max_pool(A4, ksize = [1,num_flights,num_flights,1], strides = [1,1,1,1], padding = 'SAME')
    
    Z5 = tf.nn.conv2d(P4,W5, strides = [1,1,1,1], padding = 'SAME')
    
    A5 = tf.nn.relu(Z5)
    
    # FLATTEN
    P5 = tf.contrib.layers.flatten(A5)
    
    # FULLY-CONNECTED without non-linear activation function

    Z6 = tf.contrib.layers.fully_connected(P5, num_outputs = 15, activation_fn=None)

    return Z6

In [7]:
# Compute_cost using softmax cross entropy

def compute_cost(Z3, Y):
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    
    return cost

In [8]:
# Calculating the false and negative positive/negative existed in the prediction
# Used for latter calculation of evaluation metrics

def model_eval(predict_op, truth_label):
    counter_correct, counter_tp, counter_fn, counter_fp, counter_tn = np.zeros(5)
    total_prediction = truth_label.shape[0]
    for group in range(truth_label.shape[0]):
        if (truth_label[group] > 3 and predict_op[group] > 3):
            counter_tn += 1
        if (truth_label[group] <= 3 and predict_op[group] <= 3):
            counter_tp += 1
        if (truth_label[group] > 3 and predict_op[group] <= 3):
            counter_fp += 1
        if (truth_label[group] <= 3 and predict_op[group] > 3):
            counter_fn += 1
        if (truth_label[group] == predict_op[group]):
            counter_correct += 1
    
    return counter_correct, counter_tp, counter_fn, counter_fp, counter_tn 
        
    

In [9]:
# Calculate evaluation metrics of the model
def model_metrics(counter_tp, counter_fn, counter_fp, counter_tn):
    binary_accuracy = (counter_tp + counter_tn)/(counter_tp + counter_tn + counter_fp + counter_fn)
    binary_precision = (counter_tp)/(counter_tp + counter_fp)
    binary_recall = (counter_tp)/(counter_tp + counter_fn)
    f_1 = (2*binary_precision*binary_recall)/(binary_precision + binary_recall)
    
    return binary_accuracy, f_1

In [16]:
# Building the CNN Model

def model(X_train, Y_train, X_dev, Y_dev, learning_rate = 0.001,
          num_epochs = 60, print_cost = True):
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    (m, n_H0, n_W0, n_C0) = X_train.shape 
    n_y = Y_train.shape[1]  
    training_costs = []                                          # To keep track of the cost
    dev_costs = []
    
    # Create Placeholders of the correct shape
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y)

    # Initialize parameters
    parameters = initialize_parameters()
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    Z = forward_propagation(X, parameters)
    
    # Cost function: Add cost function to tensorflow graph
    cost = compute_cost(Z, Y)
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
    
    # Create Mini-Batches
    X_minibatches = np.array_split(X_train, int(X_train.shape[0]/128))
    Y_minibatches = np.array_split(Y_train, int(Y_train.shape[0]/128))
     
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):
            
            for minibatch in range(len(Y_minibatches)):
                _ , training_cost = sess.run([optimizer, cost], feed_dict={X: X_minibatches[minibatch], Y: Y_minibatches[minibatch]}) 
            if print_cost == True and epoch % 5 == 0:
                dev_cost = sess.run(cost, feed_dict={X: X_dev, Y: Y_dev})
                training_costs.append(training_cost)
                dev_costs.append(dev_cost)
                print ("Training Cost after epoch %i: %f" % (epoch, training_cost))
                print ("Dev Cost after epoch %i: %f" % (epoch, dev_cost))
                 
        # Plot the cost
        plt.plot(np.squeeze(training_costs))
        plt.plot(np.squeeze(dev_costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (epochs)')
        plt.title("CNN model loss")
        plt.legend(['train', 'test'], loc='upper right')
        plt.show()
        
        # Evaluate on Training set and print evaluation metrics
        counter_correct, counter_tp, counter_fn, counter_fp, counter_tn = np.zeros(5)
        for minibatch in range(len(Y_minibatches)):
            predict = tf.argmax(Z, 1)
            predict_op = predict.eval({X: X_minibatches[minibatch], Y: Y_minibatches[minibatch]})
            truth_label = np.argmax(Y_minibatches[minibatch], 1)
            counter_correct_m, counter_tp_m, counter_fn_m, counter_fp_m, counter_tn_m = model_eval(predict_op, truth_label)
            counter_tp += counter_tp_m
            counter_fn += counter_fn_m
            counter_fp += counter_fp_m
            counter_tn += counter_tn_m
            counter_correct += counter_correct_m
        train_binary_accuracy, train_f_1 = model_metrics(counter_tp, counter_fn, counter_fp, counter_tn)
        train_accuracy = counter_correct/X_train.shape[0]
        print("Train Accuracy:", train_accuracy)
        print("Train Accuracy (binary): ", train_binary_accuracy)
        print("Train F-1 (binary): ", train_f_1)

        # Evaluate on Dev set and print evaluation metrics
        predict = tf.argmax(Z, 1)
        predict_op = predict.eval({X: X_dev, Y: Y_dev})
        truth_label = np.argmax(Y_dev, 1)
        counter_correct, counter_tp, counter_fn, counter_fp, counter_tn = model_eval(predict_op, truth_label)
        dev_binary_accuracy, dev_f_1 = model_metrics(counter_tp, counter_fn, counter_fp, counter_tn)
        dev_accuracy = counter_correct/X_dev.shape[0]
        print("Dev Accuracy:", dev_accuracy)
        print("Dev Accuracy (binary): ", dev_binary_accuracy)
        print("Dev F-1 (binary): ", dev_f_1)
            
        return train_accuracy, dev_accuracy, parameters

In [ ]:
_,_, parameters = model(X_train, Y_train, X_dev, Y_dev)